In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
from collections import Counter
from collections import deque
import random

random.seed(42)

# 加载Adult数据集
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
    'hours-per-week', 'native-country', 'income'
]
adult_data = pd.read_csv(url, names=column_names, na_values=' ?')

# 数据预处理
# 处理缺失值
adult_data.dropna(inplace=True)

# 将分类数据进行编码
adult_data = pd.get_dummies(adult_data, drop_first=True)

# 分离特征和标签
X = adult_data.drop('income_ >50K', axis=1)
y = adult_data['income_ >50K']

# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
%%time

class ManualKNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for x in X_test:
            # 计算与所有训练样本的欧氏距离
            distances = np.sqrt(np.sum((self.X_train - x)**2, axis=1))
            # 获取距离最近的k个训练样本的索引
            nearest_neighbors = np.argsort(distances)[:self.k]
            # 获取这k个训练样本的标签
            nearest_labels = self.y_train.iloc[nearest_neighbors]
            # 进行投票，选择票数最多的类别作为预测结果
            y_pred.append(Counter(nearest_labels).most_common(1)[0][0])
        return np.array(y_pred)

# 创建KNN对象并进行训练
knn = ManualKNN(k=5)
knn.fit(X_train, y_train)

# 进行预测
y_pred_manual = knn.predict(X_test)

accuracy_manual = accuracy_score(y_test, y_pred_manual)
print(accuracy_manual)

0.8131940991214984
CPU times: user 31.5 s, sys: 7.82 s, total: 39.3 s
Wall time: 39.4 s


In [3]:
%%time

# KD 树节点定义
class KDNode:
    def __init__(self, data, split_dim, left, right):
        self.data = data
        self.split_dim = split_dim
        self.left = left
        self.right = right

# KD 树构建函数
def build_kdtree(points, depth=0):
    if len(points) == 0:
        return None
    
    k = points.shape[1]  # 维度数
    axis = depth % k  # 当前划分的维度

    # 根据当前维度排序，选择中位数作为划分点
    points_sorted = points[points[:, axis].argsort()]
    median_idx = len(points_sorted) // 2
    median_point = points_sorted[median_idx]

    # 递归构建左右子树
    left_child = build_kdtree(points_sorted[:median_idx], depth + 1)
    right_child = build_kdtree(points_sorted[median_idx + 1:], depth + 1)

    return KDNode(median_point, axis, left_child, right_child)

# KD 树搜索函数
def kdtree_knn_search(tree, point, k):
    def euclidean_distance(p1, p2):
        return np.sqrt(np.sum((p1 - p2) ** 2))

    if tree is None:
        return []

    k_neighbors = []
    queue = deque([tree])

    while queue:
        node = queue.popleft()

        # 计算当前节点到目标点的距离
        dist = euclidean_distance(node.data[:-1], point)
        
        # 将当前节点加入最近邻列表
        k_neighbors.append((dist, node.data[-1]))

        # 保持最近邻列表长度为 k
        k_neighbors.sort()
        k_neighbors = k_neighbors[:k]

        # 根据当前维度计算目标点与当前节点的距离
        axis_dist = point[node.split_dim] - node.data[node.split_dim]

        # 优先搜索更接近目标点的子树
        if axis_dist <= 0:
            if node.left:
                queue.append(node.left)
        else:
            if node.right:
                queue.append(node.right)

    return k_neighbors

# KNN 分类器
def knn_classifier(X_train, y_train, X_test, k):
    predictions = []
    tree = build_kdtree(np.column_stack((X_train, y_train)))

    for test_point in X_test:
        neighbors = kdtree_knn_search(tree, test_point, k)
        neighbor_labels = [neighbor[1] for neighbor in neighbors]
        prediction = Counter(neighbor_labels).most_common(1)[0][0]
        predictions.append(prediction)

    return predictions

# 设置 KNN 参数
k_neighbors = 5

# 使用 KD 树加速的 KNN 分类
predictions = knn_classifier(X_train, y_train, X_test, k_neighbors)

# 计算准确率
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.7807061163600199
CPU times: user 544 ms, sys: 59.2 ms, total: 603 ms
Wall time: 611 ms
